In [1]:
import pandas as pd
import ast

In [2]:
def makeframe_fromlst(inp, ind=None):
    df = pd.DataFrame(inp)
    if ind:
        df.sort_values(ind, inplace = True)
        df.set_index(ind, inplace = True)
    return df

In [3]:
def makeframe_fromtxt(inp, ind=None):
    df = pd.DataFrame(list(ast.literal_eval(open(inp, 'r').read())))
    if ind:
        df.sort_values(ind, inplace = True)
        df.set_index(ind, inplace = True)
    return df

In [4]:
def makelst_fromsplittxt(inp1, inp2):
    dl = list(ast.literal_eval(open(inp1, 'r').read()))
    dl.extend(list(ast.literal_eval(open(inp2, 'r').read())))
    return dl

In [5]:
tmddf1 = makeframe_fromtxt("tmd_data_01_on_04_03_2021.txt")

In [6]:
tmddf1.drop_duplicates('id', inplace = True)

In [7]:
tmddf1

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,/hQ4pYsIbP22TMXOUdSfC2mjWrO0.jpg,None,0,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",,2,tt0094675,fi,Ariel,...,1988-10-21,0,73.0,"[{'english_name': 'German', 'iso_639_1': 'de',...",Released,,Ariel,False,6.8,129
1,False,/l94l89eMmFKh7na2a1u5q67VgNx.jpg,None,0,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",,3,tt0092149,fi,Varjoja paratiisissa,...,1986-10-17,0,74.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,,Shadows in Paradise,False,7.2,127
2,False,/u0zMKKpEdDWpOKmFW2sLbKKICJH.jpg,None,4000000,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",,5,tt0113101,en,Four Rooms,...,1995-12-09,4257354,98.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,False,5.7,1916
3,False,/5aXp2s4l6g5PcMMesIj63mx8hmJ.jpg,None,21000000,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",,6,tt0107286,en,Judgment Night,...,1993-10-15,12136938,110.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Don't move. Don't whisper. Don't even breathe.,Judgment Night,False,6.5,184
4,False,None,None,42000,"[{'id': 99, 'name': 'Documentary'}]",http://www.lifeinloops.com,8,tt0825671,en,Life in Loops (A Megacities RMX),...,2006-01-01,0,80.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,A Megacities remix.,Life in Loops (A Megacities RMX),False,7.4,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381827,False,None,None,0,"[{'id': 99, 'name': 'Documentary'}]",None,539608,None,en,Inside the California Missions,...,2017-01-01,0,58.0,[],Released,,Inside the California Missions,False,0.0,0
381828,False,None,None,0,[],None,539609,tt0135562,fr,Le mystère Imberger,...,1935-04-12,0,NaN,[],Released,,Le mystère Imberger,False,0.0,0
381829,False,None,None,0,[],None,539610,tt6766436,es,La hora de la merienda,...,2017-06-18,0,17.0,[],Released,,La hora de la merienda,False,0.0,0
381830,False,None,None,0,[],None,539611,None,es,El reflejo del mamífero,...,,0,28.0,[],Released,,The Mammalian Reflex,False,0.0,0


In [8]:
imdbmfdf = pd.read_csv('imdb_mov_filtered.zip')

In [12]:
tmddf1.to_csv('tmmdfp1_04_03_2021.zip')

In [ ]:
import dask.dataframe as dd
from dask.delayed import delayed
import dask.bag as db
import glob

filenames = glob.glob('tmd_data_*_on_04_03_2021.txt')

'''
The key to getting around the CParse error was using sep=None
Came from this post
http://stackoverflow.com/questions/37505577/cparsererror-error-tokenizing-data
'''

# custom saver function for dataframes using newfilenames
def reader(filename):
    return pd.read_csv(filename,sep=None)

# build list of delayed pandas csv reads; then read in as dask dataframe

dfs = [delayed(reader)(fn) for fn in filenames]
df = dd.from_delayed(dfs)


'''
This is the final step.  The .compute() code below turns the 
dask dataframe into a single pandas dataframe with all your
files merged. If you don't need to write the merged file to
disk, I'd skip this step and do all the analysis in 
dask. Get a subset of the data you want and save that.  
'''
df = df.reset_index().compute()
df.to_csv('./test.csv')